In [ ]:
import torch
from tuned_lens.nn.lenses import TunedLens
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from tuned_lens.scripts.train_loop_latent import RelatedCollator
from tuned_lens.scripts.lens_latent import RelatedDataset
from tuned_lens.plotting import plot_lens
#import ipywidgets as widgets
from plotly import graph_objects as go
import json
import os

In [ ]:
device = torch.device('cpu')
loadModel = False
modelName = 'gpt2'
# modelName = 'sshleifer/tiny-gpt2'
# To try a diffrent modle / lens check if the lens is avalible then modify this code
model = AutoModelForCausalLM.from_pretrained(modelName)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(modelName)
if loadModel:
    tuned_lens = TunedLens.load(modelName, map_location=device)
else:
    tuned_lens = TunedLens(model)
tuned_lens = tuned_lens.to(device)


In [ ]:
dataDir = "datasets/"
with open(dataDir +"related.json", "r") as f: 
    relatedJson = json.load(f)
        
tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_side='left', return_special_tokens_mask=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

dataset = RelatedDataset(relatedJson, tokenizer)
collator = RelatedCollator(tokenizer, dataset.pad_to_multiple_of)
dl = DataLoader(dataset, batch_size=2, collate_fn=collator, shuffle=True)





In [ ]:
for prompt, response, related in dl:
    len_related_words = len(related['input_ids'])
    
    for i in range(len_related_words): 
        hidden_lps, responseOutput = plot_lens.get_lens_stream(model, tokenizer, tuned_lens, related_ids = related['input_ids'][i], related_at_mask=related['attention_mask'][i])
        #print(hidden_lps.shape)
        #print(responseOutput.shape)
    
    # for each word put it in the modified plot lens to make an appropriate return 
    #response_masked_input = response['input_ids'] * response['attention_mask']
   
            